In [1]:
import logging
import sys, os
from dotenv import load_dotenv

load_dotenv('../.env')

jinaai_api_key = os.environ.get("JINAAI_API_KEY")
groq_api_key = os.environ.get("GROQ_API_KEY")

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
import textwrap

In [2]:
from llama_index.embeddings.jinaai import JinaEmbedding

Settings.embed_model = JinaEmbedding(
    api_key=jinaai_api_key,
    embed_batch_size=1536,
    model="jina-embeddings-v2-base-en",
)

In [3]:
from llama_index.llms.groq import Groq

Settings.llm = Groq(model="llama3-70b-8192", api_key=groq_api_key)


/home/aizat/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

print("Document ID:", documents[0].doc_id)

Document ID: d0473658-fcc7-4c68-aee0-03b1604bddf3


In [5]:
# Create an index over the documnts
from llama_index.core import StorageContext
import os


vector_store = MilvusVectorStore(
    uri='http://127.0.0.1:9091',
    dim=1536,
    overwrite=True,
    #enable_sparse=True,
    #hybrid_ranker="RRFRanker",
    #hybrid_ranker_params={"k": 60},
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


Failed to create new connection using: 9541ba080a584147888b5cba1cdcc9b2


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 127.0.0.1:9091, illegal connection params or server unavailable)>

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

In [ ]:
query_engine = index.as_query_engine()


In [ ]:
response = query_engine.query("What did the author learn?")
#print(textwrap.fill(str(response), 100))

In [ ]:
response = query_engine.query("What was a hard moment for the author?")
print(textwrap.fill(str(response), 100))

In [ ]:
from FlagEmbedding import BGEM3FlagModel
from typing import List
from llama_index.vector_stores.milvus.utils import BaseSparseEmbeddingFunction


class ExampleEmbeddingFunction(BaseSparseEmbeddingFunction):
    def __init__(self):
        self.model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=False)

    def encode_queries(self, queries: List[str]):
        outputs = self.model.encode(
            queries,
            return_dense=False,
            return_sparse=True,
            return_colbert_vecs=False,
        )["lexical_weights"]
        return [self._to_standard_dict(output) for output in outputs]

    def encode_documents(self, documents: List[str]):
        outputs = self.model.encode(
            documents,
            return_dense=False,
            return_sparse=True,
            return_colbert_vecs=False,
        )["lexical_weights"]
        return [self._to_standard_dict(output) for output in outputs]

    def _to_standard_dict(self, raw_output):
        result = {}
        for k in raw_output:
            result[int(k)] = raw_output[k]
        return result

In [ ]:
vector_store = MilvusVectorStore(
    dim=1536,
    overwrite=True,
    enable_sparse=True,
    sparse_embedding_function=ExampleEmbeddingFunction(),
    hybrid_ranker="RRFRanker",
    hybrid_ranker_params={"k": 60},
)